# TRATAMIENTO DE VARIABLES

In [1]:
# some imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings('ignore')

In [2]:
import nbimporter
import importlib

In [3]:
import Functions_for_FMP as fffmp

In [4]:
importlib.reload(fffmp)
import Functions_for_FMP as fffmp

## Carga de datos

In [5]:
data_clean='FMP_clean'
FMP = pd.read_pickle(data_clean)

In [6]:
fffmp.check_NaNs(FMP)

No hay missing values en el dataset :)


In [7]:
FMP

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

In [8]:
fffmp.Print_features(FMP)

Today Features

id
target
home_team_name
away_team_name
match_date
league_name
league_id
is_cup
home_team_coach_id
away_team_coach_id
home_team_median_rating
away_team_median_rating


Historic Features

home/away_team_history_match_date_i
home/away_team_history_is_play_home_i
home/away_team_history_is_cup_i
home/away_team_history_goal_i
home/away_team_history_opponent_goal_i
home/away_team_history_rating_i
home/away_team_history_opponent_rating_i
home/away_team_history_coach_i
home/away_team_history_league_id_i


## Tratamiento de variables

**1) RATING DIFFERENCE**

El *rating* es una variable calculada por el proveedor de los datos (Octosport) y corresponde a la fuerza relativa pronosticada **antes** del partido. En la práctica permite definir un "favorito a ganar".

Aqui entonces haré una disgreción: a la hora de analizar un problema, puede ser interesante darle un peso de "relevancia" a cada dato que vemos. En termino de partidos, o deportes, puede ser que ciertos partidos sean una anomalía, y por tanto no tenga tanto sentido considerarlos. En el marco de este *dataset*, una anomalía sería un partido con victoria para un equipo cuando el *rating* diferencial era negativo. 

Por el momento calcularemos entonces el *rating* diferencial.

In [9]:
for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_rating_diff_{}'.format(i)
        ranting=t+'_team_history_rating_{}'.format(i)
        op_ranting=t+'_team_history_opponent_rating_{}'.format(i)
        FMP[new_label]=FMP[ranting]-FMP[op_ranting]
        FMP=FMP.drop(columns=[ranting,op_ranting])

Al ser relativo a cada partido, esta cantidad no tiene porqué estar correlacionado con el partido a predecir.

Distinto es el caso del rating medio del local y visitante, que esperamos que nos de información de la fortaleza relativa de los equipos para el partido a predecir. Pero para que esto sirva, tendremos que verificar que ambos equipos juegan la misma liga. Como ya dijimos, solo en ese caso tiene sentido. Caso contrario pondremos $0$.

In [10]:
for t in ['home','away']:
    C=[]
    for i in range(1,11):
        C.append(t+'_team_history_league_id_{}'.format(i))
    FMP[t+'_liga']=FMP[C].mode(axis=1)[0]
        
h='home_team_median_rating'
a='away_team_median_rating'
FMP['Rating_diff']=(FMP[h]-FMP[a])*(FMP['away_liga']==FMP['home_liga']).astype('int')*(FMP['home_liga']!=-1).astype('int')        

FMP=FMP.drop(columns=[h,a,'home_liga','away_liga'])

**2) DIFERENCIA DE GOL**

En este segundo paso, lo que haremos es quedarnos con la diferencia de goles en cada partido como información relevate:

In [11]:
for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_goal_diff_{}'.format(i)
        goal=t+'_team_history_goal_{}'.format(i)
        op_goal=t+'_team_history_opponent_goal_{}'.format(i)
        
        FMP[new_label] = FMP[goal]-FMP[op_goal]
        
        FMP=FMP.drop(columns=[goal,op_goal])

**3) CONTINUIDAD DEL ENTRENADOR**

Los cambios de entrenador muchas veces tienen un efecto animico que se traslada a los resultados, trataremos de capturar eso con una variable. La nueva variable o *fetaure* valdrá $0$ cuando haya un cambio de entrenador, o $1$ si se mantiene el mismo.

(Para el partido más antiguo presupondré continuidad)

In [12]:
for t in ['home','away']:
    for i in range(1,10):
        
        new_label=t+'_coach_continuity_{}'.format(i)
        
        c=t+'_team_history_coach_{}'.format(i)
        a=t+'_team_history_coach_{}'.format(i+1)
        
        FMP[new_label] = ( (FMP[c]==FMP[a]) | ( (FMP[c]==-1) | (FMP[a]==-1)    ) ).astype('int')
    
    #para el partido a predecir
    new_label=t+'_coach_continuity'
    c=t+'_team_coach_id'
    a=t+'_team_history_coach_{}'.format(1)
    
    FMP[new_label] = ( (FMP[c]==FMP[a]) | ( (FMP[c]==-1) | (FMP[a]==-1)    ) ).astype('int')
    
    #para el partido más antíguo i=10
    new_label=t+'_coach_continuity_{}'.format(10)
    FMP[new_label] = np.ones(len(FMP)).astype('int')
    
    
for t in ['home','away']:
    for i in range(1,11):
        c=t+'_team_history_coach_{}'.format(i)
        FMP=FMP.drop(columns=[c])
    c=t+'_team_coach_id'
    FMP=FMP.drop(columns=[c])

**4) ACUMULACION DE PARTIDOS Y CANSANCIO RELATIVO**

En este punto intentaremos tener en cuenta el efecto de la acumulación de partidos. Eso no solo puede tener efectos en el cansancio de los jugadores sino también puede obligar a una mayor rotación de jugadores. De esta forma, esperamos que pueda impactar de alguna manera en el rendimiento de los equipos.

Lo que nos importará aqui no será el valor absoluto, sino el diferencial entre ambos equipos. Despues de todo, si ambos estan expuestos al mismo ritmo de juegos, este factor ya pierde importancia.

Calcularemos el descanso medio entre partidos (en días) desde el partido $i=1$, $i=3$ y desde el $i=10$. Asigaremos un peso relatio de $0.6$, $0.3$ y $0.1$, respectivamente. 

Finalmente, la resta de ambos nos dará el descanso relativo. De tomar un valor positivo, indicará que el Local (*home*) tiene menos desgste físico.

In [13]:
d0='match_date'

t='home'
d1_h=t+'_team_history_match_date_1'
d2_h=t+'_team_history_match_date_3'
d3_h=t+'_team_history_match_date_10'

t='away'
d1_a=t+'_team_history_match_date_1'
d2_a=t+'_team_history_match_date_3'
d3_a=t+'_team_history_match_date_10'


new_label='Descanso_entre_partidos_relativo'

FMP[new_label]= (0.1*(((FMP[d0]-FMP[d3_h]).dt.days)/10)+0.3*(((FMP[d0]-FMP[d2_h]).dt.days)/3)+0.6*(((FMP[d0]-FMP[d1_h]).dt.days)))-(0.1*(((FMP[d0]-FMP[d3_a]).dt.days)/10)+0.3*(((FMP[d0]-FMP[d2_a]).dt.days)/3)+0.6*(((FMP[d0]-FMP[d1_a]).dt.days)))

**5) ANTIGUEDAD DEL PARTIDO**

La importancia, incluencia o peso de un resultado del pasado puede depender de que tan lejos en el tiempo está. Partidos de temporadas anteriores, o incluso de antes de un corte por fecha FIFA pueden tener menor importancia. 

Para tratar de capturar ello, utilizaremos las fechas de los partidos para calcular (en días) cuanto tiempo pasó desde que se jugaron.

In [14]:
d='match_date'
for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_relevance_{}'.format(i)
        c=t+'_team_history_match_date_{}'.format(i)
        
        FMP[new_label]= (FMP[d]-FMP[c]).dt.days

Y ahora si, vamos a eliminar las fechas del dataset:

In [15]:
d='match_date'
FMP=FMP.drop(columns=[d])
for t in ['home','away']:
    for i in range(1,11):
        c=t+'_team_history_match_date_{}'.format(i)
        FMP=FMP.drop(columns=[c])

**6) RESULTADO NOMINAL DE LOS PARTIDOS DEL HISTORIAL**

De los partidos $i\in[1,10]$ tenemos la diferencia de gol, pero no el resultado nominal (i.e.: empate (E), victoria (V), derrota (D). Agregaremos esta variable porque nos será de utilidad a la hora de evaluar modelos.

**Nota:** La idea no es utilizar la diferencia de goles y este resultado nominal al mismo tiempo. Esta variables estan altamente correlacionadas, por lo cual no tendría sentido. Simplemente la queremos tener para ver cuanto mejor es tener la diferencia de gol, o si al final con tener el resultado nominal es suficiente.

Crearemos esta variable directamente como un one-hot encoding. Tendremos las variables $D_i$ y $V_i$, que tomaran valores $0$ o $1$. Cuando ambas son 0, queda implicito el empate. 

In [16]:
for t in ['home','away']:
    for i in range(1,11):
        
        new_label_V=t+'_outcome_V_{}'.format(i)
        new_label_D=t+'_outcome_D_{}'.format(i)
    
        x1=t+'_goal_diff_{}'.format(i)
        
        FMP[new_label_V]=(FMP[x1]>0).astype('int')
        FMP[new_label_D]=(FMP[x1]<0).astype('int')

**7) PRTIDOS AMISTOSOS**

Una cosa que puyede ser interesante es ponerle labels a los partidos amistosos. Es sabido que estos partidos pueden no ser muy representativos. Primero busquemos los ids de esas competiciones:

In [17]:
print('Ligas amistosas: ',FMP.league_name[FMP['league_name'].str.contains('riend')].unique())
Friendly_competitions_ids=FMP.league_id[FMP['league_name'].str.contains('riend')].unique()

Ligas amistosas:  ['Club Friendlies' 'Friendly International' 'Friendly International Women'
 'Club Friendlies Women' 'Friendly International U19'
 'Friendly International U23']


Y ahora, creamos esa *feature*:

In [18]:
for t in ['home','away']:
    for i in range(1,11):
        league_id=t+'_team_history_league_id_{}'.format(i)
        amistoso=t+'_is_friendly_{}'.format(i)
        FMP[amistoso]=(FMP[league_id].isin(Friendly_competitions_ids)).astype('int')

In [19]:
league_id='league_id'
amistoso='is_friendly'
FMP[amistoso]=(FMP[league_id].isin(Friendly_competitions_ids)).astype('int')

**8) DIFERENCIA DE RATING CORREGIDA**

Como fuimos comentando, el rating es un valor calculado a priori. lo que intentaremos luego, es generar una diferencia de *rating* a posteriori (corregida por el resultado del partido). No es claro si la usaremos, pero puede ser de utilidad.

In [20]:
R=[]
G=[]
for t in ['home','away']:
    for i in range(1,11):
        g=t+'_goal_diff_{}'.format(i)
        r=t+'_rating_diff_{}'.format(i)
        G.append(g)
        R.append(r)

m_g=np.mean(FMP[G].to_numpy())
m_r=np.mean(FMP[R].to_numpy())

std_g=np.std(FMP[G].to_numpy())
std_r=np.std(FMP[R].to_numpy())

for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_corrected_rating_diff_{}'.format(i)
        g=t+'_goal_diff_{}'.format(i)
        r=t+'_rating_diff_{}'.format(i)
        
        FMP[new_label] = np.abs((FMP[r]-m_r)/std_r)*((FMP[g]-m_g)/std_g)
        

Luego:
* El signo de este rating depende del resultado
* Y el valor del producto entre lo esperado y lo observado, todo normalizado.

**9) ELIMINACION DE *FEATURES***

Finalmenete eliminaremos las variables o *features* que no tienen valor predictivo.

In [21]:
FMP=FMP.drop(columns=['home_team_name','away_team_name','league_name','league_id'])
for t in ['home','away']:
    for i in range(1,11):
        l=t+'_team_history_league_id_{}'.format(i)
        FMP=FMP.drop(columns=[l])

**Guardamos datos**

In [22]:
data_clean='FMP_final'
FMP.to_pickle(data_clean)  # where to save it, usually as a .pkl
#df = pd.read_pickle(Clean_Data)

In [23]:
display(FMP)

,id,target,is_cup,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,home_rating_diff_1,home_rating_diff_2,home_rating_diff_3,home_rating_diff_4,home_rating_diff_5,home_rating_diff_6,home_rating_diff_7,home_rating_diff_8,home_rating_diff_9,home_rating_diff_10,away_rating_diff_1,away_rating_diff_2,away_rating_diff_3,away_rating_diff_4,away_rating_diff_5,away_rating_diff_6,away_rating_diff_7,away_rating_diff_8,away_rating_diff_9,away_rating_diff_10,Rating_diff,home_goal_diff_1,home_goal_diff_2,home_goal_diff_3,home_goal_diff_4,home_goal_diff_5,home_goal_diff_6,home_goal_diff_7,home_goal_diff_8,home_goal_diff_9,home_goal_diff_10,away_goal_diff_1,away_goal_diff_2,away_goal_diff_3,away_goal_diff_4,away_goal_diff_5,away_goal_diff_6,away_goal_diff_7,away_goal_diff_8,away_goal_diff_9,away_goal_diff_10,home_coach_continuity_1,home_coach_continuity_2,home_coach_continuity_3,home_coach_continuity_4,home_coach_continuity_5,home_coach_continuity_6,home_coach_continuity_7,home_coach_continuity_8,home_coach_continuity_9,home_coach_continuity,home_coach_continuity_10,away_coach_continuity_1,away_coach_continuity_2,away_coach_continuity_3,away_coach_continuity_4,away_coach_continuity_5,away_coach_continuity_6,away_coach_continuity_7,away_coach_continuity_8,away_coach_continuity_9,away_coach_continuity,away_coach_continuity_10,Descanso_entre_partidos_relativo,home_relevance_1,home_relevance_2,home_relevance_3,home_relevance_4,home_relevance_5,home_relevance_6,home_relevance_7,home_relevance_8,home_relevance_9,home_relevance_10,away_relevance_1,away_relevance_2,away_relevance_3,away_relevance_4,away_relevance_5,away_relevance_6,away_relevance_7,away_relevance_8,away_relevance_9,away_relevance_10,home_outcome_V_1,home_outcome_D_1,home_outcome_V_2,home_outcome_D_2,home_outcome_V_3,home_outcome_D_3,home_outcome_V_4,home_outcome_D_4,home_outcome_V_5,home_outcome_D_5,home_outcome_V_6,home_outcome_D_6,home_outcome_V_7,home_outcome_D_7,home_outcome_V_8,home_outcome_D_8,home_outcome_V_9,home_outcome_D_9,home_outcome_V_10,home_outcome_D_10,away_outcome_V_1,away_outcome_D_1,away_outcome_V_2,away_outcome_D_2,away_outcome_V_3,away_outcome_D_3,away_outcome_V_4,away_outcome_D_4,away_outcome_V_5,away_outcome_D_5,away_outcome_V_6,away_outcome_D_6,away_outcome_V_7,away_outcome_D_7,away_outcome_V_8,away_outcome_D_8,away_outcome_V_9,away_outcome_D_9,away_outcome_V_10,away_outcome_D_10,home_is_friendly_1,home_is_friendly_2,home_is_friendly_3,home_is_friendly_4,home_is_friendly_5,home_is_friendly_6,home_is_friendly_7,home_is_friendly_8,home_is_friendly_9,home_is_friendly_10,away_is_friendly_1,away_is_friendly_2,away_is_friendly_3,away_is_friendly_4,away_is_friendly_5,away_is_friendly_6,away_is_friendly_7,away_is_friendly_8,away_is_friendly_9,away_is_friendly_10,is_friendly,home_corrected_rating_diff_1,home_corrected_rating_diff_2,home_

In [24]:
fffmp.Print_features(FMP)

Today Features

id
target
is_cup
Rating_diff
home_coach_continuity
away_coach_continuity
Descanso_entre_partidos_relativo
is_friendly


Historic Features

home/away_team_history_is_play_home_i
home/away_team_history_is_cup_i
home/away_rating_diff_i
home/away_goal_diff_i
home/away_coach_continuity_i
home/away_relevance_i
home/away_outcome_V_i
home/away_outcome_D_i
home/away_is_friendly_i
home/away_corrected_rating_diff_i


In [25]:
FMP.describe()

,id,is_cup,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,home_rating_diff_1,home_rating_diff_2,home_rating_diff_3,home_rating_diff_4,home_rating_diff_5,home_rating_diff_6,home_rating_diff_7,home_rating_diff_8,home_rating_diff_9,home_rating_diff_10,away_rating_diff_1,away_rating_diff_2,away_rating_diff_3,away_rating_diff_4,away_rating_diff_5,away_rating_diff_6,away_rating_diff_7,away_rating_diff_8,away_rating_diff_9,away_rating_diff_10,Rating_diff,home_goal_diff_1,home_goal_diff_2,home_goal_diff_3,home_goal_diff_4,home_goal_diff_5,home_goal_diff_6,home_goal_diff_7,home_goal_diff_8,home_goal_diff_9,home_goal_diff_10,away_goal_diff_1,away_goal_diff_2,away_goal_diff_3,away_goal_diff_4,away_goal_diff_5,away_goal_diff_6,away_goal_diff_7,away_goal_diff_8,away_goal_diff_9,away_goal_diff_10,home_coach_continuity_1,home_coach_continuity_2,home_coach_continuity_3,home_coach_continuity_4,home_coach_continuity_5,home_coach_continuity_6,home_coach_continuity_7,home_coach_continuity_8,home_coach_continuity_9,home_coach_continuity,home_coach_continuity_10,away_coach_continuity_1,away_coach_continuity_2,away_coach_continuity_3,away_coach_continuity_4,away_coach_continuity_5,away_coach_continuity_6,away_coach_continuity_7,away_coach_continuity_8,away_coach_continuity_9,away_coach_continuity,away_coach_continuity_10,Descanso_entre_partidos_relativo,home_relevance_1,home_relevance_2,home_relevance_3,home_relevance_4,home_relevance_5,home_relevance_6,home_relevance_7,home_relevance_8,home_relevance_9,home_relevance_10,away_relevance_1,away_relevance_2,away_relevance_3,away_relevance_4,away_relevance_5,away_relevance_6,away_relevance_7,away_relevance_8,away_relevance_9,away_relevance_10,home_outcome_V_1,home_outcome_D_1,home_outcome_V_2,home_outcome_D_2,home_outcome_V_3,home_outcome_D_3,home_outcome_V_4,home_outcome_D_4,home_outcome_V_5,home_outcome_D_5,home_outcome_V_6,home_outcome_D_6,home_outcome_V_7,home_outcome_D_7,home_outcome_V_8,home_outcome_D_8,home_outcome_V_9,home_outcome_D_9,home_outcome_V_10,home_outcome_D_10,away_outcome_V_1,away_outcome_D_1,away_outcome_V_2,away_outcome_D_2,away_outcome_V_3,away_outcome_D_3,away_outcome_V_4,away_outcome_D_4,away_outcome_V_5,away_outcome_D_5,away_outcome_V_6,away_outcome_D_6,away_outcome_V_7,away_outcome_D_7,away_outcome_V_8,away_outcome_D_8,away_outcome_V_9,away_outcome_D_9,away_outcome_V_10,away_outcome_D_10,home_is_friendly_1,home_is_friendly_2,home_is_friendly_3,home_is_friendly_4,home_is_friendly_5,home_is_friendly_6,home_is_friendly_7,home_is_friendly_8,home_is_friendly_9,home_is_friendly_10,away_is_friendly_1,away_is_friendly_2,away_is_friendly_3,away_is_friendly_4,away_is_friendly_5,away_is_friendly_6,away_is_friendly_7,away_is_friendly_8,away_is_friendly_9,away_is_friendly_10,is_friendly,home_corrected_rating_diff_1,home_corrected_rating_diff_2,home_correct

In [26]:
fffmp.check_NaNs(FMP)

No hay missing values en el dataset :)
